In [12]:
from time import time as ltime0316
from random import random as rnd0316

class ProgressBar():
    __fill = '█'

    def __init__(self, iterations, barLength = 40):
        self.__iterations = iterations
        self.__counter = 0
        self.__start = ltime0316()
        self.__end = False
        self.__total_time = 0
        self.__frequency = 1
        self.__frequencies = []
        self.__freq10 = 1
        self.__bar_length = barLength-1
        self.__last_counter = 0
        self.__last_time = self.__start
    
    def __fill_fun(self, val):
        if val==0.0: return '█'
        if val<0.125: return '▏'
        if val<0.25: return '▎'
        if val<0.375: return '▍'
        if val<0.5: return '▌'
        if val<0.625: return '▋'
        if val<0.75: return '▊'
        if val<0.875: return '▉'
        return '█'

    def __update(self):
        this_time = ltime0316()
        deltaIter = self.__counter - self.__last_counter
        deltaTime = this_time - self.__last_time
        self.__total_time = this_time - self.__start
        self.__frequency = self.__freq(deltaIter / deltaTime)
        self.__freq10 = self.__frequency // 10
        if self.__freq10 == 0: self.__freq10 = 1
        self.__print_bar()
        self.__last_counter = self.__counter
        self.__last_time = this_time

    def next(self):
        self.__counter += 1
        if not self.__end:
            if self.__counter == self.__iterations:
                self.__end = True
                self.__update()
                return
            if self.__frequency > 100000:
                if 10/self.__frequency<rnd0316():
                    self.__update()
            elif self.__frequency<self.__iterations:
                if self.__counter % self.__freq10 == 0:
                    self.__update()
            elif self.__counter <= 3:
                self.__update()

    def reset(self, iterations, barLength = 40):
        self.__init__(iterations, barLength)
    
    def __freq(self, lastFreq):
        if len(self.__frequencies)==10:
            div = 55
            self.__frequencies.pop(0)
        elif len(self.__frequencies) == 0:
            self.__frequencies.append(lastFreq)
            return lastFreq
        else:
            div = 0
            for i in range(len(self.__frequencies)):
                div+=len(self.__frequencies)-i
        self.__frequencies.append(lastFreq)
        weight = 11 - len(self.__frequencies)
        tot = 0
        for f in self.__frequencies:
            tot += (weight * f)
            weight += 1
        freq = tot/div
        if freq > 10: return int(freq)
        return round(freq, 2)

    def __eta(self, freq):
        eta = int((self.__iterations-self.__counter) / freq)
        mins = ""
        if eta >= 60:
            mins = eta//60
            mins = str(mins) + "mins "
            eta = eta%60
        return mins + str(eta)
    
    def __last_output(self, bar):
        if self.__total_time >= 60:
            mins = self.__total_time//60
            secs = int(self.__total_time%60)
            totalTime = str(mins) +" mins " + str(secs) + " seconds"
        else:
            totalTime = str(int(self.__total_time)) + " seconds"
        return f'\r{bar} [100.00]% (total time: {totalTime})'+(20)*" "

    def __print_bar(self):
        freq = self.__frequency
        eta = self.__eta(freq)
        if freq%1==0:
            frequency = str(freq)
        else:
            frequency = "{:.2f}".format(freq)
        percent = 100*(self.__counter / float(self.__iterations))
        spercent = "{:.2f}".format(percent)
        filledLength = self.__bar_length * self.__counter // self.__iterations
        factor = (self.__bar_length * self.__counter / self.__iterations) - (self.__bar_length * self.__counter // self.__iterations)
        # bar = '|' + (self.__fill * filledLength) + '-' * (self.__bar_length - filledLength) + '|'
        bar = '|' + (self.__fill * (filledLength)) + self.__fill_fun(factor) + ' ' * (self.__bar_length - filledLength) + '|'
        if not self.__end:
            print(f'\r{bar} [{spercent}]% (eta: {eta}s, frequency: {frequency}/s)'+7*" ", end = '', flush = True)
        else:
            print(self.__last_output(bar))

In [16]:
from time import time, sleep
inizio = time()

tot = 5678
bar = ProgressBar(tot)
i=0
while i<tot:
    bar.next()
    if i<tot/2:
        sleep(0.001)
    else:
        sleep(0.002)
    
    i += 1
print(time()-inizio)
bar.reset(tot)
i=0
while i<tot:
    if i<tot/2:
        sleep(0.001)
    else:
        sleep(0.002)
    bar.next()
    i += 1

|▏                                       | [0.05]% (eta: 0s, frequency: 39247/s)          

KeyboardInterrupt: 

In [70]:
tot = 5678000
bar = ProgressBar(tot)
i=0
while i<tot:
    i += 1

In [15]:
bar.reset(50000000)
i=0
for i in range(50000000):
    bar.next()
    if i>25000000:
        sleep(1)

|█▎                                      | [2.89]% (eta: 4mins 21s, frequency: 185763/s)          

KeyboardInterrupt: 